In [ ]:
import odr_env
import json

# Find all raman files in the provided datarecord list
csv_files = {}
for dr_id in _odr_datarecord_list:
    # If this datarecord has files that have been uploaded to it...
    file_list = odr_env.getFileList(dr_id)

    if ( len(file_list) > 0 ):
        # TODO - probably need a better way to get datarecord name easily
        dr_data = odr_env.getDatarecordData(dr_id)
        dr_name = dr_data['datarecords'][0]['datarecord_name']
        sol = dr_data['datarecords'][0]['external_id']

        # Organize by sol
        csv_files[sol] = {}

        # Append all file ids to this list...should only be one file
        for file_id, file_data in file_list.items():
            csv_files[sol] = {'file_id': int(file_id), 'target': dr_name}

#print( json.dumps(csv_files, indent=4) )

In [ ]:
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np

# Create a new plot
fig = plt.figure(figsize=(15, 7), dpi=200)
ax = fig.add_subplot(111)


# Define the average soil composition of mars for a few compounds
average_soil_labels = ['Na2O', 'MgO', 'Al2O3', 'SiO2',
                       'P2O5', 'SO3', 'Cl', 'K2O',
                       'CaO', 'TiO2', 'Cr2O3', 'MnO',
                       'FeO', 'Ni', 'Zn', 'Br']
average_soil_values = [2.59, 9.04, 9.31, 44.65,
                       0.80, 3.69, 0.55, 0.44,
                       7.20, 0.98, 0.52, 0.42,
                       19.73, 0.0482, 0.0237, 0.0029]    # last three values are typically reported as 482, 237, 29


# Number of csv files dictates x coordinates...
num_files = len(csv_files)

# Define the X locations for the groups
ind = np.arange( len(average_soil_values) )
# Want a bit of whitespace between compounds
width = 0.85 / num_files

# Calculate the kludge factor so each of the series of bars lines up properly
tmp = ''
if ( (num_files % 2) == 0 ):
    # even
    tmp = num_files // 2 - 0.5
else:
    # odd
    tmp = num_files // 2

adjust = np.linspace(-tmp, tmp, num=num_files)
adjust = adjust * width


# For each location with APXS data...
labels = []
rects = []
axis_num = 1
for sol, data in sorted(csv_files.items()):
    # Store a label for this location
    target = data['target']
    labels.append(target)
    
    # ...download the data for each file from ODR
    file_id = int(data['file_id'])
    file_data = odr_env.downloadFile( file_id )

    try:
        graph_data = np.genfromtxt(BytesIO(file_data), delimiter=",")    # convert bytes into array
    except:
        print( 'error in processing file ' + str(file_id) + ' from "' + data['target'] + '"' )
        continue

    # Split the 2D array into 1D arrays for graphing
    raw_data = graph_data[1:,1]    # for each sub-array after the first, get the second element of the sub-array and make a new array out of them
#    errors = graph_data[1:,2]  # do the same for the third element of the sub-array

    # Convert the raw data into a percentage variance from mars average soil
    data = []
    for i, raw_value in enumerate(raw_data):
        variance = ((raw_value / average_soil_values[i]) - 1.0) * 100.0
        data.append( variance )
    
    # Need different colors, otherwise it'll just be a mess
    line_color = 'C' + str(axis_num)
    axis_num += 1

    # Determine where the left side of each bar should be
    x_pos = ind + adjust[axis_num-2]

    # Draw the series of data on the graph
    rect = ax.bar(x_pos, data, width, color=line_color, align='center')
    rects.append( rect[0] )

# Set up the rest of the graph
ax.set_title('Percent Difference from Mars Average Soil Composition')
ax.set_xticks(ind)
ax.set_xticklabels(average_soil_labels)
ax.legend(rects, labels)

# Apparently the graph doesn't make a line where y = 0 by default
plt.axhline(0, color='black', linewidth=0.75)

plt.show()